In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [2]:
path = os.getcwd()+'/test.csv'

In [3]:
df = pd.read_csv(path)

In [4]:
df.columns.values

array(['member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term',
       'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title',
       'emp_length', 'home_ownership', 'annual_inc', 'verification_status',
       'issue_d', 'loan_status', 'pymnt_plan', 'desc', 'purpose', 'title',
       'zip_code', 'addr_state', 'dti', 'earliest_cr_line',
       'mths_since_last_record', 'pub_rec', 'revol_bal', 'revol_util',
       'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv',
       'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp',
       'total_rec_int', 'total_rec_late_fee', 'recoveries',
       'collection_recovery_fee', 'collections_12_mths_ex_med',
       'mths_since_last_major_derog', 'policy_code', 'application_type',
       'annual_inc_joint', 'dti_joint', 'verification_status_joint',
       'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m', 'open_il_6m',
       'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il',
       'il_util', 'open_rv_12m', 

In [5]:
data = df
sub_grade_mapping = {'A1':0,'A2':1,'A3':2,'A4':3,'A5':4,
                     'B1':5,'B2':6,'B3':7,'B4':8,'B5':9,
                     'C1':10,'C2':11,'C3':12,'C4':13,'C5':14,
                     'D1':15,'D2':16,'D3':17,'D4':18,'D5':19,
                     'E1':20,'E2':21,'E3':22,'E4':23,'E5':24,
                     'F1':25,'F2':26,'F3':27,'F4':28,'F5':29,
                     'G1':30,'G2':31,'G3':32,'G4':33,'G5':34}

data.sub_grade = data.sub_grade.map(sub_grade_mapping)



In [6]:
data = data.drop(['grade'],axis=1)

In [7]:
data.sub_grade.unique() 

array([ 6, 12,  8,  1, 11,  5, 23,  7, 13, 22,  4, 10,  0, 20,  3, 19, 15,
       17, 24, 18,  9, 25, 21, 14,  2, 26, 27, 16, 30, 28, 32, 29, 33, 34,
       31])

In [8]:
#term
term_mapping = {label:idx for idx,label in enumerate(set(data['term']))}  
data.term = df.term.map(term_mapping) 


In [9]:
#emp_length
emp_length_mapping = {'10+ years':10,'9 years':9,'8 years':8,'7 years':7,'6 years':6,
                       '5 years':5,'4 years':4,'3 years':3,'2 years':2,'1 year':1,'< 1 year':0}
data.emp_length = df.emp_length.map(emp_length_mapping) 

In [10]:
data2 = data
average_emp_length = data2.dropna(axis=0,subset=['emp_length']).emp_length.mean() 
data.emp_length. = data.emp_length.fillna(average_emp_length)


SyntaxError: invalid syntax (<ipython-input-10-7ed11d730fd6>, line 3)

In [11]:
data

,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,sub_grade,emp_title,emp_length,...,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
0,24344352,11000.0,11000.0,11000.0,0,10.15,355.72,6,Custodian,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,47900.0,NaN,NaN,NaN
1,68831083,18400.0,18400.0,18400.0,1,13.18,420.36,12,President,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,46100.0,NaN,NaN,NaN
2,72767821,8800.0,8800.0,8800.0,0,10.64,286.61,8,CPA,1.0,...,63964.0,69.9,4.0,7.0,4971.0,66.8,23100.0,1.0,0.0,5.0
3,1539914,8500.0,8500.0,8500.0,0,6.62,260.99,1,FCEOC,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,68359086,1000.0,1000.0,1000.0,0,12.59,33.50,11,Plumber,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,21500.0,NaN,NaN,NaN
5,57325151,20000.0,20000.0,20000.0,0,8.18,628.39,5,Function Chemist,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,28700.0,NaN,NaN,NaN
6,69574516,4625.0,4625.0,4625.0,0,19.99,171.86,23,Operations Manager,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,4200.0,NaN,NaN,NaN
7,8075891,5500.0,5500.0,5500.0,0,11.55,181.50,7,RLSD,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,3000.0,NaN,NaN,NaN
8,57485220,14000.0,14000.0,14000.0,1,13.99,325.69,13,New Business & Sales Manager,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,12600.0,NaN,NaN,NaN
9,54025412,26000.0,26000.0,26000.0,1,19.19,677.18,22,Registered Nurse,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,43739.0,NaN,NaN,NaN


In [249]:
df.funded_amnt.astype('string')
df.loc[:,['funded_amnt','funded_amnt_inv',]][150:-150]
df.loc[:,'mths_since_last_major_derog'].count()
df.loc[:,['dti_joint','application_type',]].head(3)
data = df.loc[:,['member_id','dti_joint','application_type']]
for index, row in data.iterrows():
    if row['application_type'] != 'INDIVIDUAL':
        print(row['application_type'],row['dti_joint'])

0         11000.0
1         18400.0
2          8800.0
3          8500.0
4          1000.0
5         20000.0
6          4625.0
7          5500.0
8         14000.0
9         26000.0
10        12000.0
11         1500.0
12        12000.0
13        12500.0
14        24000.0
15         6525.0
16         7200.0
17        22800.0
18         7475.0
19        10000.0
20        24025.0
21        10000.0
22        10000.0
23         8000.0
24        10000.0
25        30000.0
26         3600.0
27         2550.0
28         3000.0
29        32400.0
           ...   
177446    15625.0
177447     7925.0
177448     8800.0
177449    15000.0
177450     4000.0
177451     2250.0
177452    25000.0
177453    10000.0
177454     3600.0
177455    25000.0
177456    29475.0
177457    10000.0
177458     4800.0
177459    24000.0
177460    35000.0
177461    20000.0
177462    10000.0
177463    14550.0
177464    10400.0
177465    22675.0
177466    15275.0
177467    17500.0
177468     5500.0
177469     1500.0
177470    